# Midiprocessing

In [1]:
from adaptivetuning import Midiprocessing

<IPython.core.display.Javascript object>

A Midiprocessing objects receives midi messages from a controller (e.g. a midi-keyboard) or reads a midi file. On every note-on and note-off message, it calls some given callback function.

In [2]:
m = Midiprocessing()

You can stop a Midiprocessor by calling its stop method or by setting a threading.Event you pass when calling play_port or play_file, but when we are working in a single thread, the midi processing will block the thread and prevent us from stopping it.

For a simple test of Midiprocessing in a single thread, we can specify a maximal number of notes it will process, after that it will stop automatically.

In [3]:
m.max_notes = 5

We have to specify what to do with the received messages.

In [4]:
m.note_on_callback = lambda pitch, amp: print('Note on:', pitch, amp)
m.note_off_callback = lambda pitch: print('Note off', pitch)
m.stop_callback = lambda: print('Stop')

Let's read a midi file and play it first 5 notes!

In [5]:
m.read_file("midi_files/BWV_0227.mid")
m.play_file()

Note on: 71 0.7874015748031497
Note on: 67 0.7874015748031497
Note on: 64 0.7874015748031497
Note on: 52 0.7874015748031497
Note off 71
Note off 67
Note off 64
Note off 52
Note on: 71 0.7874015748031497
Stop


We can control an Audiogenerator object with a Midiprocessing object by connecting the callbacks with the corresponding methods of the Audiogenerator. The register_audiogenerator method can do that for us.

In [13]:
import sc3nb
sc = sc3nb.startup()
#sc = None

Starting sclang...
Done.
Starting osc communication...
This sc3nb sc instance is at port: 57130
Done.
Registering UDP callback...
Done.
Sclang started on non default port: 57121
Booting server...
Done.
-> sc3nb started


In [14]:
from adaptivetuning import Audiogenerator
a = Audiogenerator(sc = sc)
a.set_synth_def_with_dict(Audiogenerator.presets['piano'])

In [55]:
m.register_audiogenerator(a)
m.max_notes = 30

In [20]:
m.read_file("midi_files/BWV_0227.mid")
m.play_file()

Let's hear the first two bars of Bach's 'Jesu meine Freude' with a stretched timbre and a stretched scale!

In [51]:
a.partials_pos = {'method': 'harmonic', 'nr_partials': 10, 'octave': 2.8}
a.partials_amp = [0.6**i for i in range(10)]
a.scale.octave_interval = 2.8
a.scale.tune_all_equal_temperament()  # 12TET with streched octave

In [59]:
m.play_file()

If we want to play something ourselves, we need some kind of midi port, for example from a midi keyboard. If a midi keyboard is connected to your machine, you should see if in the following list:

In [53]:
m.get_port_names()

['MPK mini']

Let's play exactly 30 notes with our midi device.

In [58]:
m.port_name = 'MPK mini'
m.play_port()